In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(2)
import random
import io
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras import layers, models, regularizers
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
from keras.regularizers import l1_l2 
from keras.layers import *
import pandas as pd
from keras.optimizers import Adam
from keras.initializers import GlorotUniform 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau  # Added ReduceLROnPlateau
from keras import models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras import layers
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16, ResNet50, ResNet101
from PIL import Image, ImageChops, ImageEnhance
from tensorflow.keras.applications import EfficientNetB4
import os
import itertools

In [2]:
# Define paths
directory = '/kaggle/input/ai-vs-human-generated-dataset'
train_data = '/kaggle/input/ai-vs-human-generated-dataset/train_data'
train_csv = '/kaggle/input/ai-vs-human-generated-dataset/train.csv'
test_data = '/kaggle/input/ai-vs-human-generated-dataset/test_data_v2'
test_csv = '/kaggle/input/ai-vs-human-generated-dataset/test.csv'

In [3]:
# Define image size and ELA quality
image_size = (48, 48)  # Desired image size
# ela_quality = 90  # ELA quality parameter

In [4]:
# Load the training CSV file
train_df = pd.read_csv(train_csv)
test_df= pd.read_csv(test_csv)

In [5]:
# Remove the 'train_data/' prefix from the file_name column
train_df['file_name'] = train_df['file_name'].str.replace('train_data/', '', regex=False)
# Remove the 'train_data/' prefix from the file_name column
test_df['id'] = test_df['id'].str.replace('test_data_v2/', '', regex=False)

In [6]:
#def convert_to_ela_image(image, quality):
#     """Convert an image to ELA (Error Level Analysis) format."""
#     # Ensure the image is in uint8 format and has the correct shape
#     if image.dtype != np.uint8:
 #        image = (image * 255).astype(np.uint8)  # Scale to [0, 255] and convert to uint8
    
#     # Convert the NumPy array to a PIL Image
  #   original_image = Image.fromarray(image)
    
#     # Save the original image to a BytesIO object with a specific quality
   #  temp_buffer = io.BytesIO()
    # original_image.save(temp_buffer, 'JPEG', quality=quality)
     #temp_buffer.seek(0)
    
#     # Open the compressed image from the BytesIO object
     #temp_image = Image.open(temp_buffer)
    
#     # Compute the ELA image
     #ela_image = Image.fromarray(np.abs(np.array(original_image) - np.array(temp_image)))
     #return ela_image


In [7]:
# # Function to preprocess an image (convert to ELA, resize, normalize)
def ela_preprocessing(image):
#     """Preprocess an image by converting it to ELA, resizing, and normalizing."""
#     # Convert the image to ELA format
     ela_image = convert_to_ela_image(image, ela_quality)
    
#     # Resize the image
     resized_image = ela_image.resize(image_size)
    
#     # Normalize the image to [0, 1]
     return np.array(resized_image) / 255.0

In [8]:
# Split the dataset into two categories
category_0 = train_df[train_df['label'] == 0]  # Images with label 0
category_1 = train_df[train_df['label'] == 1]  # Images with label 1

In [9]:
print(len(category_0))
print(len(category_1))

39975
39975


In [10]:
# Randomly sample 20,000 images from each category
num_samples = 39975
category_0_sampled = category_0.sample(n=num_samples, random_state=42)  # Sample 20,000 from category 0
category_1_sampled = category_1.sample(n=num_samples, random_state=42)  # Sample 20,000 from category 1

# Combine the sampled data into a balanced DataFrame
balanced_df = pd.concat([category_0_sampled, category_1_sampled])

# Shuffle the balanced DataFrame
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [11]:
# Split the balanced dataset into training and validation sets
train_data_df, val_data_df = train_test_split(balanced_df, test_size=0.2, random_state=42)

In [12]:
# Convert the 'label' column to strings
train_data_df['label'] = train_data_df['label'].astype(str)
val_data_df['label'] = val_data_df['label'].astype(str)

In [13]:
# Trim whitespace from filenames
test_df['id'] = test_df['id'].str.strip()

In [14]:
epochs = 50
batch_size = 128

In [15]:
# Define data augmentation for training
train_datagen = ImageDataGenerator(
    #preprocessing_function=ela_preprocessing,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define validation and test generators (without augmentation)
val_datagen = ImageDataGenerator(
    #preprocessing_function=ela_preprocessing  
)

test_datagen = ImageDataGenerator(
    #preprocessing_function=ela_preprocessing  
)
# Create generators for training
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data_df,
    directory=train_data,
    x_col='file_name',
    y_col='label',
    target_size=image_size,  
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=True  
)
# Create generators for validation
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data_df,
    directory=train_data,
    x_col='file_name',
    y_col='label',
    target_size=image_size, 
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=False  
)
# Create the test generator again after checks
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_data,
    x_col='id',  
    y_col=None,  
    target_size=image_size, 
    batch_size=batch_size,
    class_mode=None, 
    shuffle=False 
)

Found 63960 validated image filenames belonging to 2 classes.
Found 15990 validated image filenames belonging to 2 classes.
Found 5540 validated image filenames.


In [16]:
# Print generator details
print("Training generator:", train_generator)
print("Validation generator:", val_generator)
print("Test generator:", test_generator)

Training generator: <keras.src.legacy.preprocessing.image.DataFrameIterator object at 0x7a75e1ad17b0>
Validation generator: <keras.src.legacy.preprocessing.image.DataFrameIterator object at 0x7a75e1ad3f10>
Test generator: <keras.src.legacy.preprocessing.image.DataFrameIterator object at 0x7a7653d94760>


In [17]:
 #def build_InceptionV3_model():
     #base_model = InceptionV3(weights=None, include_top=False, input_shape=(224, 224, 3))  # Changed to ResNet101

#     # Add new layers on top of the model
     #x = base_model.output  # Output of the base model
     #x = GaussianNoise(0.1)(x)  # Add Gaussian noise with a standard deviation of 0.1
     #x = GlobalAveragePooling2D()(x)  # Global average pooling layer
     #x = Dense(1024, activation='relu', kernel_initializer=GlorotUniform(), kernel_regularizer=l1_l2(l1=0.02, l2=0.04))(x)  # Dense layer with ReLU activation and L2 regularization
     #x = Dropout(0.4)(x)  # Dropout layer for regularization
     #predictions = Dense(2, activation='softmax')(x)  # Output layer with softmax activation for multi-class classification

#     # Define the model
     #model = Model(inputs=base_model.input, outputs=predictions)  # Combined model

#     # Freeze base layers
     #for layer in base_model.layers:
    #     layer.trainable = True  # Freeze base layers for training

   #  return model

In [18]:
def build_Sequential_model():
    model = keras.Sequential([
        keras.layers.Conv2D(32,(3,3), activation='relu', input_shape = (48,48,3)),
        keras.layers.MaxPool2D((2,2)),
        keras.layers.Dropout(0.2),
        
        keras.layers.Conv2D(64,(3,3), activation='relu'),
        keras.layers.MaxPool2D((2,2)),
        keras.layers.Dropout(0.2),
        
        keras.layers.Conv2D(128,(3,3), activation='relu'),
        keras.layers.MaxPool2D((2,2)),
        keras.layers.Dropout(0.2),
        
        keras.layers.Conv2D(256,(3,3), activation='relu'),
        keras.layers.MaxPool2D((2,2)),
        keras.layers.Dropout(0.2),
            
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
         keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(2, activation='softmax')  
    ])
    return model

In [19]:
lr_schedule = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor validation loss
    factor=0.1,  # Reduce learning rate by a factor of 0.1
    patience=3,  # Wait for 3 epochs before reducing the learning rate
    min_lr=1e-6  # Minimum learning rate
)


In [20]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Stop training if no improvement for 5 epochs
    restore_best_weights=True
)

In [21]:
import keras
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Set the learning rate
learning_rate = 1e-4

model = build_Sequential_model()
model.summary()
# Compile the model with the specified learning rate
model.compile(optimizer=Adam(learning_rate=learning_rate), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 46, 46, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 2, 2, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 432,210 (1.65 MB)

 Trainable params: 432,210 (1.65 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Train the model using the generators
hist = model.fit(
    train_generator,  # Use the train_generator created with flow_from_dataframe
    steps_per_epoch=len(train_data_df) // batch_size,  # Number of batches per epoch
    epochs=epochs,
    validation_data=val_generator,  # Use the val_generator created with flow_from_dataframe
    validation_steps=len(val_data_df) // batch_size,  # Number of validation batches
    callbacks=[early_stopping, lr_schedule],  # Callbacks for early stopping and learning rate scheduling
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


499/499 ━━━━━━━━━━━━━━━━━━━━ 585s 1s/step - accuracy: 0.5504 - loss: 1.2335 - val_accuracy: 0.7253 - val_loss: 0.5517 - learning_rate: 1.0000e-04
Epoch 2/50
  1/499 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.7422 - loss: 0.4964

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


499/499 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7422 - loss: 0.4964 - val_accuracy: 0.7119 - val_loss: 0.5588 - learning_rate: 1.0000e-04
Epoch 3/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 267s 529ms/step - accuracy: 0.7293 - loss: 0.5438 - val_accuracy: 0.6232 - val_loss: 0.5857 - learning_rate: 1.0000e-04
Epoch 4/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/step - accuracy: 0.7500 - loss: 0.5381 - val_accuracy: 0.6610 - val_loss: 0.5530 - learning_rate: 1.0000e-04
Epoch 5/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 273s 541ms/step - accuracy: 0.7839 - loss: 0.4710 - val_accuracy: 0.5842 - val_loss: 0.6194 - learning_rate: 1.0000e-05
Epoch 6/50
499/499 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step - accuracy: 0.7109 - loss: 0.5363 - val_accuracy: 0.6695 - val_loss: 0.5723 - learning_rate: 1.0000e-05


In [23]:
model.save('model_Sequential_run1.h5')

In [24]:
import numpy as np

# Make predictions using the test generator
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)

# Convert predictions to class labels (assuming 2 classes)
predicted_labels = np.argmax(predictions, axis=1)  # Get the index of the highest probability

# Add predictions to the test DataFrame
test_df['label'] = predicted_labels

44/44 ━━━━━━━━━━━━━━━━━━━━ 191s 4s/step


In [25]:
# # Make predictions using the test generator
# predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)

# # Convert predictions to binary labels
# predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]

# # Add predictions to the test DataFrame
# test_df['label'] = predicted_labels

In [26]:
# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'label': test_df['label'].astype(int)  # Convert predictions to integers
})

In [27]:
# # Save the submission file, remove rows with NaN labels
# submission_df = submission_df.dropna(subset=['label'])
submission_df.to_csv('submission_sh_Sequential.csv', index=False)
print("Submission file 'submission.csv' created successfully.")

Submission file 'submission.csv' created successfully.
